In [1]:
import PIL
from PIL import Image
import numpy as np
import os 
import random
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt

from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jan/a

In [2]:
height = 128
width = 128
depth = 3
inputShape = (height, width, depth)

In [3]:
def create_model():
    
    model = Sequential()
    model.add(Conv2D(3,(6,6), padding="same", input_shape = inputShape))   #, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(6, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(2))
    model.add(Activation("sigmoid"))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model 


In [4]:
def build_cnn(activation = 'relu',
              dropout_rate = 0.2,
              optimizer = 'Adam'):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3),
              activation=activation,
              input_shape=inputShape))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(2, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model

In [5]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [16]:
param_grid = {
              'epochs':[1,2,3],
              'batch_size':[16]
              #'epochs' :              [100,150,200],
              #'batch_size' :          [32, 128],
              #'optimizer' :           ['Adam', 'Nadam'],
              #'dropout_rate' :        [0.2, 0.3],
              #'activation' :          ['relu', 'elu']
             }

In [17]:
model = KerasClassifier(build_fn = build_cnn, verbose=0)

In [18]:
gs = GridSearchCV(estimator=model,param_grid=param_grid,cv=3,n_jobs=-1,scoring='neg_mean_squared_error')

In [1]:
fitted = gs.fit(trainX, trainY)

NameError: name 'gs' is not defined

In [82]:
model, pred = algorithm_pipeline(trainX, testX, trainY, testY, model, 
                                        param_grid, cv=5, scoring_fit='neg_log_loss')

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:   10.0s remaining:    8.8s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   10.2s finished


In [83]:
print(model.best_score_)
print(model.best_params_)

nan
{'batch_size': 16, 'epochs': 1}


In [7]:
dirs = ['/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/no_porosity',
        '/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/porosity']

length = 2000
image_paths = []
labels = []
data = []

In [8]:
for d in dirs: 
    cur_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d,path)
        cur_paths.append(full_path)
        
    random.seed(42)
    random.shuffle(cur_paths)
    cur_paths = cur_paths[:length]
    
    image_paths.append(cur_paths)        

image_paths = [val for sublist in image_paths for val in sublist]
random.seed(42)
random.shuffle(image_paths)

######
for path in image_paths:
    label = path.split(os.path.sep)[-2]
    labels.append(label)
    
    array = np.load(path)
    img = Image.fromarray(array)
    img = img.resize((height, width), PIL.Image.LANCZOS)
    array_resized_flattened = np.array(img)
    data.append(array_resized_flattened)
    
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
trainY = np.hstack((trainY, 1- trainY))

testY = lb.transform(testY)
testY = np.hstack((testY, 1- testY))

In [10]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(3000, 128, 128, 3)
(3000, 2)
(1000, 128, 128, 3)
(1000, 2)


In [11]:
trainY[2]

array([1, 0])

In [9]:
aug = ImageDataGenerator(vertical_flip= True,horizontal_flip=True)

In [45]:
#creating the model 
model = KerasClassifier(build_fn = create_model,batch_size=10, epochs=10)

In [32]:
scores = cross_val_score(model, trainX, trainY, cv=5)

Epoch 1/1
600/600 [==============================] - 1s 1ms/step
Epoch 1/1
600/600 [==============================] - 1s 1ms/step
Epoch 1/1
600/600 [==============================] - 1s 1ms/step
Epoch 1/1
600/600 [==============================] - 1s 1ms/step
Epoch 1/1


KeyboardInterrupt: 

In [ ]:
scores

In [15]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.71 (+/- 0.03)


In [52]:
#defining grid search parameters 
batch_size = [8,16]
epochs = [5,10]
param_grid = dict(batch_size = batch_size, epochs = epochs)

In [53]:
param_grid

{'batch_size': [8, 16], 'epochs': [5, 10]}

In [47]:
param_grid

{'batch_size': [8, 16], 'epochs': [5, 10]}

In [48]:
grid = GridSearchCV(estimator = model, param_grid = param_grid,scoring='accuracy', n_jobs = -1, cv = 3)

In [49]:
grid_result = grid.fit(trainX, trainY)

Epoch 1/5
3000/3000 [==============================] - 7s 2ms/step - loss: 0.6702 - acc: 0.6298
Epoch 2/5
3000/3000 [==============================] - 5s 2ms/step - loss: 0.5928 - acc: 0.6815
Epoch 3/5
3000/3000 [==============================] - 4s 1ms/step - loss: 0.5834 - acc: 0.6788
Epoch 4/5
3000/3000 [==============================] - 5s 2ms/step - loss: 0.5588 - acc: 0.7030
Epoch 5/5
3000/3000 [==============================] - 4s 1ms/step - loss: 0.5469 - acc: 0.7113


In [50]:
grid_result.cv_results_

{'mean_fit_time': array([1.15439479, 1.60136779, 1.36918624, 1.8485721 ]),
 'std_fit_time': array([0.14745191, 0.13793936, 0.39483136, 0.10245835]),
 'mean_score_time': array([0., 0., 0., 0.]),
 'std_score_time': array([0., 0., 0., 0.]),
 'param_batch_size': masked_array(data=[8, 8, 16, 16],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[5, 10, 5, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 8, 'epochs': 5},
  {'batch_size': 8, 'epochs': 10},
  {'batch_size': 16, 'epochs': 5},
  {'batch_size': 16, 'epochs': 10}],
 'split0_test_score': array([nan, nan, nan, nan]),
 'split1_test_score': array([nan, nan, nan, nan]),
 'split2_test_score': array([nan, nan, nan, nan]),
 'mean_test_score': array([nan, nan, nan, nan]),
 'std_test_score': array([nan, nan, nan, nan]),
 'rank_test_score': array([1, 2, 3, 4], dtype=int32)}

In [30]:
grid_result.best_score_

nan

In [31]:
grid_result.cv_results_['mean_test_score']

array([nan, nan, nan, nan])

In [24]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: nan using {'batch_size': 8, 'epochs': 25}
nan (nan) with: {'batch_size': 8, 'epochs': 25}
nan (nan) with: {'batch_size': 8, 'epochs': 50}
nan (nan) with: {'batch_size': 16, 'epochs': 25}
nan (nan) with: {'batch_size': 16, 'epochs': 50}


In [ ]:
model2 = SmallVGGNet.build(width=128, height=128, depth=3, classes=len(lb.classes_))

In [ ]:
INIT_LR = 0.001
EPOCHS = 80
BS = 32

print("[INFO] training network...")
opt = Adam(lr = INIT_LR)#, decay=INIT_LR / EPOCHS) # SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model2.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
H = model2.fit_generator(aug.flow(trainX, trainY, batch_size=BS),validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,epochs=EPOCHS)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model2.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
#plt.savefig(args["plot"])

In [ ]:
model2.summary()

In [ ]:
model2.save('ownCNN')